@Author: Pooja
@Date: 21-11-2024
@Last Modified by: Pooja
@Last Modified time: 21-11-2024
@Title: Python program to perform Gen AI tasks Inferring and Expanding using Gemini API

In [13]:
import os
import csv
from dotenv import load_dotenv
import google.generativeai as genai

Load environment variables

In [14]:
load_dotenv(dotenv_path='C:/Users/pooja konduri/OneDrive/Desktop/BridgeLabs/GenAI_tasks/key.env')
genai.configure(api_key=os.getenv('GEMINI_API_KEY'))

Configure Gemini AI

In [16]:
model = genai.GenerativeModel(
    model_name="gemini-1.5-flash",
    generation_config={
        "temperature": 0.9,
        "top_p": 0.95,
        "top_k": 64,
        "max_output_tokens": 8192,
    },
)

 Read reviews from the input file 

In [17]:
def read_reviews(file_path, delimiter="---END OF REVIEW---"):
    """
    Step 1: Read reviews from the input file split by the given delimiter.
    """
    with open('C:/Users/pooja konduri/OneDrive/Desktop/BridgeLabs/GenAI_tasks/input/reviews.txt', 'r') as file:
        return [review.strip() for review in file.read().split(delimiter) if review.strip()]

Extract reviews

In [18]:
def extract_product_and_review(review_text):
    """
    Step 2: Extract product name and review content.
    """
    lines = review_text.split("\n")
    product = next((line.split(":")[1].strip() for line in lines if line.startswith("Product:")), "")
    review = next((line.split(":")[1].strip() for line in lines if line.startswith("Review:")), "")
    return product, review

 infer sentiment 

In [19]:

def infer_sentiment_and_themes(review, chat_session):
    """
    Step 3: Use Gemini to infer sentiment and extract themes.
    """
    sentiment = chat_session.send_message(f"Determine the sentiment of this review: {review}").text
    themes = chat_session.send_message(f"Identify key themes in this review: {review}").text
    return sentiment, themes


 expanded review using inferred insights

In [20]:

def expand_review(review, sentiment, themes, chat_session):
    """
    Step 4: Generate an expanded review using inferred insights.
    """
    prompt = (
        f"Expand the following review using the given sentiment and themes:\n\n"
        f"Review: {review}\n"
        f"Sentiment: {sentiment}\n"
        f"Themes: {themes}"
    )
    return chat_session.send_message(prompt).text

 Process a single review to extract, infer, and expand.

In [21]:
def process_review(review_text, chat_session):
    """
    Step 5: Process a single review to extract, infer, and expand.
    """
    product, review = extract_product_and_review(review_text)
    sentiment, themes = infer_sentiment_and_themes(review, chat_session)
    expanded = expand_review(review, sentiment, themes, chat_session)
    return product, review, sentiment, themes, expanded

saving to csv file

In [22]:
def save_reviews_to_csv(data, file_name):
    """
    Step 6: Save processed reviews into a CSV file.
    """
    with open(file_name, 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(["Product", "Original Review", "Sentiment", "Themes", "Expanded Review"])
        writer.writerows(data)

In [24]:
def main():
    # Initialize chat session
    chat_session = model.start_chat()
    
    # Inferring and Expanding Pipeline
    reviews = read_reviews('reviews.txt')[:3]   # Step 1
    processed_data = [process_review(review, chat_session) for review in reviews]  # Steps 2-5
    save_reviews_to_csv(processed_data, 'transformed_reviews.csv')  # Step 6

if __name__ == '__main__':
    main()